<a href="https://colab.research.google.com/github/oscarkemuel/ConcurrentEuler/blob/master/C%C3%B3pia_de_Laborat%C3%B3rio_5_Qu%C3%A3o_eficazes_s%C3%A3o_os_ITS_implementados_com_GenAI%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratório 5 - Quão eficazes são os ITS implementados com GenAI?

<br>

Os Sistemas de Tutoria Inteligente (ITS) são agentes inteligentes que especificam *o que* ensinar e *como* ensinar de forma adaptativa. Por ser um conceito amplo, comporta diferentes abordagens de implementação.

Neste laboratório, você irá criar uma implementação simplificada de um ITS completo utilizando IA Generativa (GenAI) e refletir sobre a eficácia dessa abordagem. São fornecidos códigos preliminares para apoiar sua jornada.

Ao concluir a implementação, em `📝 Tarefa`, você irá responder à algumas perguntas por meio de testes com dois domínios distintos, refletindo sobre o título deste Laboratório.

> Atividade de laboratório do curso IMD0511.
>
> Autor: Adelson de Araujo (adelson.araujo@imd.ufrn.br).

<br>




### Parte 1 - Adquirir familiaridade com as operações básicas de desenvolvimento com GenAI.

Você irá explorar a biblioteca `google.generativeai` com o código demonstrado em sala de aula: [Tutorial de Introdução ao Desenvolvimento de IA Generativa com Python.ipynb](https://colab.research.google.com/drive/1fUFhU919QXAjCupxpfVA6Xsvs_wMxiiE?usp=sharing).

**Instruções**:

1. Configurar o ambiente.
2. Usar `generate_content()` para uma interação simples.
3. Usar `start_chat()` para criar uma conversa com histórico.
4. Inspecionar `chat.history` para entender como o histórico funciona.


In [ ]:
import json
from google.colab import userdata

# Configure sua API Key
# IMPORTANTE: Nunca coloque sua chave diretamente no código em produção.
# Use variáveis de ambiente ou o gerenciador de "Secrets" do seu notebook (ex: Google Colab).
# Para este exemplo, vamos supor que você a colou aqui:
API_KEY = userdata.get('API_KEY')

if API_KEY is not None:
  print("API_KEY loaded successfully.")
else:
  print("API_KEY not found in environment variables.")

API_KEY loaded successfully.


In [ ]:
import google.generativeai as genai

genai.configure(api_key=API_KEY)

# Escolher o modelo que vamos usar
llm = genai.GenerativeModel('models/gemini-2.5-flash')

print(llm.generate_content("ping").text)
print(llm.generate_content("já falei ping?").text)

Pong! How can I help you today?


In [ ]:
chat = llm.start_chat()

resposta = chat.send_message("ping")
print(resposta.text)
resposta2 = chat.send_message("já falei ping?")
print(resposta2.text)

print("---Historico---")
print(chat.history)

### Parte 2 - Desenvolver o ciclo adaptativo de um ITS por meio de GenAI.


#### **Instruções**:



* Em [📄 Código-fonte do ITS], são fornecidas as funções já pré-implementadas e outras a serem implementadas.
* Implemente as funções `etapa_3_avaliacao_interacao` e `etapa_7_atualizar_aluno`, teste seu funcionamento.

* Rode o ITS na seção [🔁 Controle e execução do ITS]

* Refine os prompts utilizados nas outras funções para aprimorar/customizar seu ITS.

* Ao concluir, siga para a [📝 Tarefa].


#### 📄 **Código-fonte do ITS**

In [ ]:
# --- Modelo de Domínio ---
def etapa_0_prep_modelo_dominio(llm, dominio, n_topicos=10, audiencia="4o ano do Ensino Fundamental"):
    """
    Define o Modelo de Domínio em tópicos, cada um com explicação, pré-requisito, e exercício.
    """
    prompt_dominio = f"""
    Você é especializado no domínio `{dominio}`.
    Pense nos {n_topicos} tópicos mais relevantes para a audiência "{audiencia}".
    Para cada tópico, precisará de (1) uma explicação espontâneamente simples,
    (2) a lista dos outros tópicos que são pré-requisitos deste tópico e (3) um
    exercício para avaliar a compreensão do tópico.
    As explicações e exercícios devem ser auto-contidos, não referenciando outras frases.
    O formato da sua saída deverá ser um json válido.
    """
    # Exemplo para servir de base para construção automática de novas bases de conhecimento
    exemplo_base_conhecimento = {
        "soma_fracoes": {
            "explicacao": "Para somar frações, primeiro encontre um denominador comum. Depois, some os numeradores.",
            "pre_requisitos": ["denominador_comum"],
            "exercicio": "Quanto é 1/2 + 1/4?"
        },
        "denominador_comum": {
            "explicacao": "O denominador comum é um múltiplo compartilhado pelos denominadores de duas ou mais frações.",
            "pre_requisitos":[],
            "exercicio": "Qual é o menor denominador comum para 1/3 e 1/5?"
        },
        "subtracao_fracoes": {
            "explicacao": "A subtração de frações segue a mesma lógica da soma: encontre um denominador comum e depois subtraia os numeradores.",
            "pre_requisitos": ["denominador_comum", "soma_fracoes"],
            "exercicio": "Quanto é 3/4 - 1/4?"
        }
    }
    # Transformar o dicionário acima em uma string mais legível:
    exemplo_base_conhecimento = json.dumps(exemplo_base_conhecimento, indent=4)
    prompt_dominio += f"\n Exemplo:\n {exemplo_base_conhecimento}"
    # Gerar modelo de domínio
    modelo_dominio = llm._______________(prompt_dominio).text
    return modelo_dominio


In [ ]:
# Testando

modelo_dominio = etapa_0_prep_modelo_dominio(_____, _____, _____, _____) # como chamar essa função corretamente?
print(modelo_dominio)

In [ ]:
# --- Modelo do Aluno ---
def etapa_0_inicializar_aluno(modelo_dominio):
    """
    Inicializando o modelo do aluno como nível de maestria em cada um dos todos tópicos.
    """
    modelo_aluno = {topico: "iniciante" for topico in modelo_dominio}
    return modelo_aluno

In [ ]:
# Testando

modelo_aluno = etapa_0_inicializar_aluno(_____) # como chamar essa função corretamente?
print("---Modelo do Aluno---")
print(modelo_aluno)

In [ ]:
# --- Modelo Pedagógico ---
def etapa_1_selecao_proximo_topico(llm, modelo_aluno):
    """
    Usa o LLM para decidir qual a próxima tarefa (Macro-Adaptação).
    """
    prompt_selecao_tarefa = f"""
    Você é um Sistema de Tutoria Inteligente.
    Decida o próximo tópico a ser ensinado com base no Modelo do Aluno a seguir:
    **Modelo do Aluno**:\n {json.dumps(modelo_aluno, indent=4)}\n
    Pense passo a passo para tomar sua decisão:
    1. Analise o histórico para entender o que o aluno já aprendeu.
    2. Com base na sua análise, identifique o próximo conceito mais lógico a ser ensinado, somente ensinando um tópico se os pré-requisitos forem julgados como aprendidos.
    3. Retorne sua decisão em um formato JSON com a chave "proximo_topico" (elemento presente nos Tópicos Disponíveis) e "raciocinio" (2-3 frases ultra-concisas).
    """
    return llm.generate_content(prompt_selecao_tarefa).text

In [ ]:
# Testando

decisao = etapa_1_selecao_proximo_topico(____, _____) # como chamar essa função corretamente?
topico = decisao['proximo_topico']
print("---Próximo tópico---")
print(decisao)

In [ ]:
# === Ainda não implementado ===

def etapa_3_avaliacao_interacao_inicial(llm, chat, modelo_aluno):
  """
  Usa o LLM para avaliar a resposta inicial e atualizar o modelo do aluno.
  """
  mensagem_usuario = chat.history[-1].text
  prompt_avaliacao = f"""
  ...
  ...
  ...
  ...
  """
  # modelo_aluno = {"topico1": "iniciante"}
  # ...
  # avaliacao = llm.generate_content(prompt_avaliacao).text
  # ...
  # modelo_aluno = {"topico1": "intermediário"}
  return avaliacao, modelo_aluno

In [ ]:
# Testando

chat = llm.start_chat()

mensagem_aluno = "ping"

resposta = chat.send_message(mensagem_aluno)

modelo_aluno, modelo_aluno = etapa_3_avaliacao_interacao_inicial(llm, chat, modelo_aluno)
print("---Modelo do Aluno (pós avaliação de interação inicial) ---")
print(modelo_aluno)

In [ ]:
def etapa_45_decidir_e_gerar_feedback(chat, exercicio):
    """
    Usa o LLM para avaliar a resposta e gerar feedback adaptativo (Micro-Adaptação).
    """
    resposta_aluno = chat.history[-1].text
    prompt_feedback = f"""
    --- Decisão de feedback ---
    Como ITS, Avalie a resposta do aluno e forneça um feedback útil.
    Exercício Proposto: *"{exercicio}"*
    Resposta do Aluno: **"{resposta_aluno}"**
    Pense passo a passo:
    1. A resposta está correta?.
    2. Se incorreta, identifique o equívoco.
    3. Gere uma dica sem dar a resposta.
    4. Formule sua resposta final para o aluno.
    ----------------------------
    Feedback:
    """
    resposta = chat.send_message(prompt_feedback)
    return resposta.text

In [ ]:
# Testando

feedback = etapa_45_decidir_e_gerar_feedback(chat, exercicio=modelo_dominio[topico]['exercicio'])
print(feedback)

In [ ]:
# === Ainda não implementado ===

# --- Modelo do Aluno ---
def etapa_7_atualizacao_pos_feedback(chat, modelo_aluno):
    """
    Atualiza o modelo do aluno com base no último ciclo de conversa.
    """
    if len(chat.history>3):
      # Atualize o modelo do aluno com  base n as últimas três mensagens do
      # historico: interacao inicial, feedback, e interacao final do ciclo.
      ciclo_interacao = chat.history[-3:]
      # ...
      # ...
      # ...
      # modelo_aluno = ...
    return modelo_aluno

In [ ]:
# Testando

modelo_aluno = etapa_7_atualizacao_pos_feedback(chat, modelo_aluno)
print(modelo_aluno)

#### **🔁 Controle e execução do ITS**

In [ ]:
def sistema_tutoria_inteligente_genai(llm, ____, ____, ____):
    """
    Inicia uma nova sessão de tutoria.
    """
    prompt_sistema = {"role": "model", "parts": ["Você é um Sistema de Tutoria Inteligente (ITS)."]}
    chat = llm.start_chat(history=[prompt_sistema])

    modelo_dominio = etapa_0_prep_modelo_dominio(...)
    modelo_aluno = etapa_0_inicializar_aluno(...)

    # Enquanto houver topicos a aprender...
    while modelo_aluno:

      # Etapa 1
      decisao = etapa_1_selecao_proximo_topico(llm, modelo_aluno)

      explicacao = modelo_dominio[decisao['proximo_topico']]['explicacao']
      exercicio = modelo_dominio[decisao['proximo_topico']]['exercicio']

      print(explicacao)
      print(exercicio)

      # Etapa 2
      mensagem_usuario = input("Usuário:")

      # Etapa 3
      modelo_aluno = etapa_3_avaliacao_interacao_inicial(...)

      # Etapas 4 e 5
      feedback = etapa_45_decidir_e_gerar_feedback(...)

      print(feedback)

      # Etapa 6
      mensagem_usuario = input("Usuário:") # etapa_7

      # Etapa 7
      modelo_aluno = etapa_7_atualizacao_pos_feedback(...)

      # Condição de parada: ter aprendido todos os tópicos
      if modelo_aluno[decisao['proximo_topico']] == 'avançado':
        # Removendo tópico já aprendido
        modelo_aluno.pop(decisao['proximo_topico'])

    return chat

In [ ]:
sistema_tutoria_inteligente_genai(...)

### Parte 3 - 📝 Tarefa

Submeter no SIGAA o link deste colab notebook prenchido, com última alteração feita antes da Data de fechamento da tarefa.

Seu notebook deve conter um relatório formulado abaixo, respondendo à seguintes perguntas:

<br>

-----------



> Nome: << SEU_NOME_AQUI >>

##### 1. Avalie a eficácia do seu ITS para apoiar a aprendizagem de três tipos de alunos: o aluno iniciante, o intermediário e o avançado.


RESPOSTA:

##### 2. Faça uma simulação de tutoria fictícia de um **aluno iniciante** (interpretado por você) nos domínios de `fotossíntese` e `coesão textual`, quais são os prós e os contras dessa abordagem de tutoria?

##### 3. Quais seriam bons próximos passos para aprimorar essa implementação de tutoria?

.